In [3]:
####################################################################################################
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target #rotulos
df = df[['sepal length (cm)', 'petal width (cm)', 'target']] # 3 colunas
df = df[df['target'].isin([0, 1])] # seleciona calsses setosa e versicolor
# separando as clounas classes 0 e 1 e features
R = [0, 1] 
X = df.iloc[:, R]  
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = LogisticRegression()
modelo.fit(X_train, y_train)
score = modelo.score(X_test, y_test)

w1 = modelo.coef_[0][0]  # Peso'sepal length'
w2 = modelo.coef_[0][1]  # Peso 'petal width'
wo = modelo.intercept_[0]  # Termo constante gerado pela reg. logistica

print(f"Pesos: {modelo.coef_}")
print(f"Intercepto: {modelo.intercept_}")
print(f"Acurácia: {score:.2f}")


Pesos: [[1.37515913 3.50507158]]
Intercepto: [-9.9889242]
Acurácia: 1.00


In [2]:
########## FUNÇÕES ##########
def ONEEXPLANATION(Vs, delta, R, Idx, Xpl, classe):
    """
    Encontrar uma PI-explicação usando um algoritmo guloso.
    Args:
        Vs: Valores da instância.
        delta: Lista ordenada de valores de delta.
        R: Limite de explicação.
        Idx: Índice atual na lista delta.
        Xpl: Conjunto de literais da explicação.
        classe: Classe atual sendo considerada.
    Returns:
        Tupla com o limite atualizado (R) e o índice atualizado (Idx).
    """
    # Verifica se Idx está dentro dos limites da lista antes de incrementá-lo
    if Idx + 1 < len(delta):
        # Se o peso da primeira feature for maior
        if abs(w1) > abs(w2):
            Idx = 0 # Escolhe a primeira feature como a mais importante
        else:
            Idx = 1 # Escolhe a segunda feature como a mais importante

        R -= delta[Idx]
        # Converte a tupla para string para que seja "hashable"
        Xpl.add(str((Idx, Vs[Idx])))  # Adiciona o literal à PI-explicação
        EXPLICAR_PI(Xpl, classe, w1, w2, df)  # Imprime ou processa a PI-explicação atual
        return R, Idx
    else:
        # Se Idx estiver fora dos limites, interrompe o loop
        return R, Idx

def ALLEXPLANATIONS(Vs, delta, threshold, w1, w2):
    """
    Enumerar todas as PI-explicações usando backtracking, considerando todas as classes.

    Args:
        Vs: Valores da instância.
        delta: Lista ordenada de valores de delta.
        threshold: Limite de explicação.
        w1: Peso para a primeira feature.
        w2: Peso para a segunda feature.

    Returns:
        Lista de tuplas com a classe e a PI-explicação para cada instância.
    """
    pi_explicacoes = []  # Lista para armazenar as PI-explicações
    for classe in range(2):  # Itera sobre as duas classes (0 e 1)
        Xpl = set()  # cria um set vazio para ramazenar os literais da pi-explica
        Idx = 0
        R = 0
        
        while Idx >= 0 and Idx < len(delta): 
            R, Idx = ONEEXPLANATION(Vs, delta, R, Idx, Xpl, classe)
            # Ajuste para garantir que R não se torne negativo
            if R < 0:
                R = 0
            # Adiciona a PI-explicação à lista, apenas se não existir na lista
            if (classe, Xpl) not in pi_explicacoes:
                pi_explicacoes.append((classe, Xpl))  # Adiciona a PI-explicação à lista
            Idx += 1 # Incrementa Idx após a chamada da função ONEEXPLANATION
    return pi_explicacoes
# print(pi_explicacoes)
# print(delta)
def EXPLICAR_PI(Xpl, classe, w1, w2, df):
    """Imprime a PI-explicação."""
    print(f"Classe: {classe}")
    print(f"PI-explicação: {Xpl}")
    for item in Xpl:
         idx, valores = eval(item)
         if idx == 0:
             print(f"  - Sepal Length ({w1}): {valores[0]} cm")
             print(f"    Valor Mínimo para Sepal Length: {df['sepal length (cm)'].min()} cm")
             print(f"  - Petal Width ({w2}): {valores[1]} cm")
             print(f"    Valor Mínimo para Petal Width: {df['petal width (cm)'].min()} cm")
         else:
             print(f"  - Sepal Length (peso - {w1}): {valores[0]} cm")
             print(f"    Valor Mínimo para Sepal Length: {df['sepal length (cm)'].min()} cm")
             print(f"  - Petal Width (peso - {w2}): {valores[1]} cm")
             print(f"    Valor Mínimo para Petal Width: {df['petal width (cm)'].min()} cm")

delta = []
for feature in df.columns[:-1]:  # Exclui a ultima coluna 'target'
    delta_feature = df[feature].max() - df[feature].min()
    delta.append(delta_feature)

# com o(threshold) mais baixo o classificador pode ficar mais rigoroso
threshold = 0 

Vs = []
for index, row in df.iterrows():
    Vs.append(list(row[:-1]))  # adiciona os pares Sepal e petal

pi_explicacoes = ALLEXPLANATIONS(Vs, delta, threshold, w1, w2)

# Cria um DataFrame com as PI-explicações
pi_explicacoes_df = pd.DataFrame(pi_explicacoes, columns=['Classe', 'PI-Explicação'])

# print(f'\n{pi_explicacoes_df}')

Classe: 0
PI-explicação: {'(1, [4.9, 0.2])'}
  - Sepal Length (peso - 1.3751591278961863): 4.9 cm
    Valor Mínimo para Sepal Length: 4.3 cm
  - Petal Width (peso - 3.505071575134543): 0.2 cm
    Valor Mínimo para Petal Width: 0.1 cm
Classe: 1
PI-explicação: {'(1, [4.9, 0.2])'}
  - Sepal Length (peso - 1.3751591278961863): 4.9 cm
    Valor Mínimo para Sepal Length: 4.3 cm
  - Petal Width (peso - 3.505071575134543): 0.2 cm
    Valor Mínimo para Petal Width: 0.1 cm


In [4]:
delta = []

for feature in df.columns[:-1]:  # Exclui a ultima coluna 'target'
    delta_feature = df[feature].max() - df[feature].min()
    delta.append(delta_feature)


print(df.columns)
print(delta_feature)
print(df[feature].max())
print(df[feature].min())
print(df[feature])
print(delta)
df

Index(['sepal length (cm)', 'petal width (cm)', 'target'], dtype='object')
1.7
1.8
0.1
0     0.2
1     0.2
2     0.2
3     0.2
4     0.2
     ... 
95    1.2
96    1.3
97    1.3
98    1.1
99    1.3
Name: petal width (cm), Length: 100, dtype: float64
[2.7, 1.7]


,sepal length (cm),petal width (cm),target
0,5.1,0.2,0
1,4.9,0.2,0
2,4.7,0.2,0
3,4.6,0.2,0
4,5.0,0.2,0
...,...,...,...
95,5.7,1.2,1
96,5.7,1.3,1
97,6.2,1.3,1
98,5.1,1.1,1


In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target #rotulos
df = df[['sepal length (cm)', 'petal width (cm)', 'target']] # 3 colunas
df = df[df['target'].isin([0, 1])] # seleciona calsses setosa e versicolor
# separando as clounas classes 0 e 1 e features
R = [0, 1] 
X = df.iloc[:, R]  
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = LogisticRegression()
modelo.fit(X_train, y_train)
score = modelo.score(X_test, y_test)

print(f"Pesos: {modelo.coef_}")
print(f"Intercepto: {modelo.intercept_}")
print(f"Acurácia: {score:.2f}\n")

w1 = modelo.coef_[0][0]  # Peso'sepal length'
w2 = modelo.coef_[0][1]  # Peso 'petal width'
wo = modelo.intercept_[0]  # Termo constante gerado pela reg. logistica

########## FUNÇÕES ##########
def ONEEXPLANATION(Vs, delta, R, Idx, Xpl, classe):
    
    """
    Encontrar uma PI-explicação usando um algoritmo guloso.

    Args:
        Vs: Valores da instância.
        delta: Lista ordenada de valores de delta.
        R: Limite de explicação.
        Idx: Índice atual na lista delta.
        Xpl: Conjunto de literais da explicação.
        classe: Classe atual sendo considerada.

    Returns:
        Tupla com o limite atualizado (R) e o índice atualizado (Idx).
    """
    # Verifica se Idx está dentro dos limites da lista antes de incrementá-lo
    if Idx + 1 < len(delta):
        # Se o peso da primeira feature for maior
        if abs(w1) > abs(w2):
            Idx = 0 # Escolhe a primeira feature como a mais importante
        else:
            Idx = 1 # Escolhe a segunda feature como a mais importante

        R -= delta[Idx]
        # Converte a tupla para string para que seja "hashable"
        Xpl.add(str((Idx, Vs[Idx])))  # Adiciona o literal à PI-explicação
        REPORTEXPLANATION(Xpl, classe, w1, w2)  # Imprime ou processa a PI-explicação atual
        return R, Idx
    else:
        # Se Idx estiver fora dos limites, interrompe o loop
        return R, Idx

def ALLEXPLANATIONS(Vs, delta, threshold, w1, w2):
    """
    Enumerar todas as PI-explicações usando backtracking, considerando todas as classes.

    Args:
        Vs: Valores da instância.
        delta: Lista ordenada de valores de delta.
        threshold: Limite de explicação.
        w1: Peso para a primeira feature.
        w2: Peso para a segunda feature.

    Returns:
        Lista de tuplas com a classe e a PI-explicação para cada instância.
    """
    pi_explicacoes = []  # Lista para armazenar as PI-explicações
    for classe in range(2):  # Itera sobre as duas classes (0 e 1)
        Xpl = set()  # cria um set vazio para ramazenar os literais da pi-explica
        Idx = 0
        R = 0
        
        while Idx >= 0 and Idx < len(delta): 
            R, Idx = ONEEXPLANATION(Vs, delta, R, Idx, Xpl, classe)
            # Ajuste para garantir que R não se torne negativo
            if R < 0:
                R = 0
            # Adiciona a PI-explicação à lista, apenas se não existir na lista
            if (classe, Xpl) not in pi_explicacoes:
                pi_explicacoes.append((classe, Xpl))  # Adiciona a PI-explicação à lista
            Idx += 1 # Incrementa Idx após a chamada da função ONEEXPLANATION
    return pi_explicacoes

def REPORTEXPLANATION(Xpl, classe, w1, w2):
    """Imprime a PI-explicação."""
    print(f"Classe: {classe}")
    print(f"PI-explicação: {Xpl}")
    EXPLICAR_PI(Xpl, w1, w2)

def EXPLICAR_PI(Xpl, w1, w2):
    """Explica os elementos da PI-explicação."""
    for item in Xpl:
         idx, valores = eval(item)
         if idx == 0:
             print(f"  - Sepal Length ({w1}): {valores[0]} cm")
             print(f"    Valor Mínimo para Sepal Length: {df['sepal length (cm)'].min()} cm")
             print(f"  - Petal Width ({w2}): {valores[1]} cm")
             print(f"    Valor Mínimo para Petal Width: {df['petal width (cm)'].min()} cm")
         else:
             print(f"  - Sepal Length ({w1}): {valores[0]} cm")
             print(f"    Valor Mínimo para Sepal Length: {df['sepal length (cm)'].min()} cm")
             print(f"  - Petal Width ({w2}): {valores[1]} cm")
             print(f"    Valor Mínimo para Petal Width: {df['petal width (cm)'].min()} cm")
       

delta = []
for feature in df.columns[:-1]:  # Exclui a ultima coluna 'target'
    delta_feature = df[feature].max() - df[feature].min()
    delta.append(delta_feature)

# com o(threshold) mais baixo o classificador pode ficar mais rigoroso
threshold = 0 

Vs = []
for index, row in df.iterrows():
    Vs.append(list(row[:-1]))  # adiciona os pares Sepal e petal

pi_explicacoes = ALLEXPLANATIONS(Vs, delta, threshold, w1, w2)

# Cria um DataFrame com as PI-explicações
pi_explicacoes_df = pd.DataFrame(pi_explicacoes, columns=['Classe', 'PI-Explicação'])

print(f'\n{pi_explicacoes_df}')

# --- Função para calcular o slack (Φ) ---
def calcular_slack(a, d, w, wo): # o slack é a folga ou tolerância que existe na decisão do classificador.
    """
    Calcula o slack para uma instância.

    Args:
        a: Valores da instância.
        d: Valores mínimos das features.
        w: Pesos do classificador.
        wo: Termo constante do classificador.

    Returns:
        O slack (Φ) para a instância.
    """
    return - (wo + sum(w[i] * (a[i] - d[i]) for i in range(len(a))))

Pesos: [[1.37515913 3.50507158]]
Intercepto: [-9.9889242]
Acurácia: 1.00

Classe: 0
PI-explicação: {'(1, [4.9, 0.2])'}
  - Sepal Length (1.3751591278961863): 4.9 cm
    Valor Mínimo para Sepal Length: 4.3 cm
  - Petal Width (3.505071575134543): 0.2 cm
    Valor Mínimo para Petal Width: 0.1 cm
Classe: 1
PI-explicação: {'(1, [4.9, 0.2])'}
  - Sepal Length (1.3751591278961863): 4.9 cm
    Valor Mínimo para Sepal Length: 4.3 cm
  - Petal Width (3.505071575134543): 0.2 cm
    Valor Mínimo para Petal Width: 0.1 cm

   Classe      PI-Explicação
0       0  {(1, [4.9, 0.2])}
1       1  {(1, [4.9, 0.2])}
